In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import numpy as np

class TabelaResumoDataScraper:
    def __init__(self, setor_financeiro, options, service,acoes,diretorio=None):
        self.setor_financeiro = setor_financeiro
        self.options = options
        self.service = service
        self.acoes = acoes
        self.diretorio =diretorio

    def navegador_get(self, acao):
        navegador = webdriver.Chrome(service=self.service, options=self.options)
        navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
        return navegador

    def obter_datas(self, navegador):
        while True:
            try:
                elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_bp"]/thead/tr/th[2]/select')
                datas = [elemento.text for elemento in elementos_data]
                if len(datas) == 1:
                    string_de_datas = datas[0]
                    datas = string_de_datas.split('\n')
                return datas
            except StaleElementReferenceException:
                continue

    def obter_dados_tabela(self, navegador, data):
        while True:
            try:
                select_element = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[2]/div[3]/table/thead/tr/th[2]/select')
                select = Select(select_element)
                select.select_by_visible_text(data)

                tabela = navegador.find_element(By.ID, 'pagina_empresa_bp')
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                lista_resumo_balanco = []
                for linha in linhas:
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_resumo_balanco.append(celula.text)

                return lista_resumo_balanco
            except StaleElementReferenceException:
                continue

    def coletar_dados_financeiros(self, navegador, datas):
        dados = {
            'datas': datas,
            'caixa_equivalentes_caixa': [np.nan] * len(datas),
            'ativo_total': [],
            'divida_curto_prazo': [np.nan] * len(datas),
            'divida_longo_prazo': [np.nan] * len(datas),
            'divida_bruta': [np.nan] * len(datas),
            'divida_liquida': [np.nan] * len(datas),
            'patrimonio_liquido': [],
            'valor_patrimonial_acao': [],
            'acoes_ordinarias': [],
            'acoes_preferenciais': [],
            'total': []
        }
        for data in datas:
            while True:
                lista_resumo_balanco = self.obter_dados_tabela(navegador=navegador, data=data)
                if lista_resumo_balanco:
                    break
            dados['ativo_total'].append(lista_resumo_balanco[1].replace('R$','').replace(',','.').strip())
            dados['patrimonio_liquido'].append(lista_resumo_balanco[3].replace('R$','').replace(',','.').strip())
            dados['valor_patrimonial_acao'].append(lista_resumo_balanco[5].replace('R$','').replace(',','.').strip())
            dados['acoes_ordinarias'].append(lista_resumo_balanco[7].replace('R$','').replace(',','.').strip())
            dados['acoes_preferenciais'].append(lista_resumo_balanco[9].replace('R$','').replace(',','.').strip())
            dados['total'].append(lista_resumo_balanco[11].replace('R$','').replace(',','.').strip())
            
        df_resumo_balanco = pd.DataFrame(dados)
        
        return df_resumo_balanco
            
    def coletar_dados_nao_financeiros(self,navegador, datas):
        dados = {
            'datas': datas,
            'caixa_equivalentes_caixa': [],
            'ativo_total': [],
            'divida_curto_prazo': [],
            'divida_longo_prazo': [],
            'divida_bruta': [],
            'divida_liquida': [],
            'patrimonio_liquido': [],
            'valor_patrimonial_acao': [],
            'acoes_ordinarias': [],
            'acoes_preferenciais': [],
            'total': []
        }

        for data in datas:
            while True:
                lista_resumo_balanco = self.obter_dados_tabela(navegador=navegador, data=data)
                if lista_resumo_balanco:
                    break
            dados['caixa_equivalentes_caixa'].append(lista_resumo_balanco[1].replace('R$','').replace(',','.').strip())
            dados['ativo_total'].append(lista_resumo_balanco[3].replace('R$','').replace(',','.').strip())
            dados['divida_curto_prazo'].append(lista_resumo_balanco[5].replace('R$','').replace(',','.').strip())
            dados['divida_longo_prazo'].append(lista_resumo_balanco[7].replace('R$','').replace(',','.').strip())
            dados['divida_bruta'].append(lista_resumo_balanco[9].replace('R$','').replace(',','.').strip())
            dados['divida_liquida'].append(lista_resumo_balanco[11].replace('R$','').replace(',','.').strip())
            dados['patrimonio_liquido'].append(lista_resumo_balanco[13].replace('R$','').replace(',','.').strip())
            dados['valor_patrimonial_acao'].append(lista_resumo_balanco[15].replace('R$','').replace(',','.').strip())
            dados['acoes_ordinarias'].append(lista_resumo_balanco[17].replace('R$','').replace(',','.').strip())
            dados['acoes_preferenciais'].append(lista_resumo_balanco[19].replace('R$','').replace(',','.').strip())
            dados['total'].append(lista_resumo_balanco[21].replace('R$','').replace(',','.').strip())
            
        df_resumo_balanco = pd.DataFrame(dados)
            
        return df_resumo_balanco
    
    def rodar_acoes(self):
        iteracao = 0
        lista_dataframes = []
        for acao in self.acoes:
            start_time = time.time()
            navegador = self.navegador_get(acao=acao)
            
            datas = self.obter_datas(navegador=navegador)
            
            if acao in self.setor_financeiro:
                df_resumo_balanco = self.coletar_dados_financeiros(navegador=navegador, datas=datas)  
                colunas = ['ativo_total', 'patrimonio_liquido']
                for col in colunas:
                    df_resumo_balanco[col] = df_resumo_balanco[col].apply(self.converter_valor)
            else:
                df_resumo_balanco = self.coletar_dados_nao_financeiros(navegador=navegador, datas=datas)  
                colunas = ['caixa_equivalentes_caixa', 'ativo_total', 'divida_liquida','divida_curto_prazo', 'divida_longo_prazo', 'divida_bruta', 'patrimonio_liquido']
                for col in colunas:
                    df_resumo_balanco[col] = df_resumo_balanco[col].apply(self.converter_valor)
                  
            df_resumo_balanco['tic'] = acao
            lista_dataframes.append(df_resumo_balanco)

            iteracao += 1
            
            if iteracao % 5 == 0:
                self.salvar_dados(lista_dataframes=lista_dataframes)
                
            end_time = time.time()

            # Calcula o tempo decorrido em segundos
            execution_time_seconds = end_time - start_time

            # Calcula o tempo decorrido em minutos
            execution_time_minutes = execution_time_seconds / 60

            # Imprime o tempo de execução em minutos
            print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")
                
        return pd.concat(lista_dataframes)

    def salvar_dados(self,lista_dataframes):
        if self.diretorio is None:
            self.diretorio = 'dados/resumo_balanco.csv'
        pd.concat(lista_dataframes).to_csv(self.diretorio)

    def converter_valor(self,valor):
        if isinstance(valor, str):
            if valor.endswith('T'):
                return float(valor.replace('T', '').replace(' ', '')) * 1_000_000
            elif valor.endswith('B'):
                return float(valor.replace('B', '').replace(' ', '')) * 1_000
            elif valor.endswith('M'):
                return float(valor.replace('M', '').replace(' ', '')) * 1
            elif valor.endswith('K'):
                return float(valor.replace('K', '').replace(' ', '')) * 0.001
            elif valor == 'NA':
                return float('nan')
        else:
            return valor

In [1]:
from StockBrazil.BalancoPatrimonial import TabelaResumoDataScraper

ImportError: cannot import name 'TabelaResumoDataScraper' from 'StockBrazil.BalancoPatrimonial' (/workspaces/fundamentalist-stock-brazil/StockBrazil/BalancoPatrimonial.py)

In [6]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    #'VVBR3', 'BTOW3','ENBR3',GNDI3,IGTA3
    'BRFS3', 'BRKM5',  'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
    'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
     'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4',  'GOAU4', 'GOLL4', 'HAPV3',
     'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
    'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3','SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
    'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'WIZC3','YDUQ3','SOMA3','BPAC11','MDIA3'
]

In [5]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3',]

In [6]:

scraper = BalancoPatrimonial.TabelaResumoDataScraper(setor_financeiro, options, service,acoes=codigos_ibovespa)

AttributeError: module 'StockBrazil.BalancoPatrimonial' has no attribute 'TabelaResumoDataScraper'

In [3]:
# Uso da classe
setor_financeiro = {'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'ITUB4', 'BPAC11', 'ITUB4', 'SANB11', 'IRBR3'}

chrome_driver_path = "/usr/bin/chromedriver"
service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

scraper = TabelaResumoDataScraper(setor_financeiro, options, service,acoes=codigos_ibovespa)

dados = scraper.rodar_acoes()

Tempo de execução da ação ABEV3: 1.12 minutos
Tempo de execução da ação AZUL4: 0.75 minutos
Tempo de execução da ação B3SA3: 1.64 minutos
Tempo de execução da ação BBAS3: 2.01 minutos
Tempo de execução da ação BBDC3: 2.17 minutos


In [4]:
dados

,datas,caixa_equivalentes_caixa,ativo_total,divida_curto_prazo,divida_longo_prazo,divida_bruta,divida_liquida,patrimonio_liquido,valor_patrimonial_acao,acoes_ordinarias,acoes_preferenciais,total,tic
0,31/03/2024,12840.00,133820.00,1650.00,2170.0,3820.0,-9030.0,86590.00,5.50,15.757.657.000,0,15.757.657.000,ABEV3
1,31/12/2023,16060.00,132640.00,1300.00,2200.0,3500.0,-12560.0,78970.00,5.01,15.753.833.000,0,15.753.833.000,ABEV3
2,30/09/2023,17410.00,137910.00,1240.00,2480.0,3720.0,-13700.0,90250.00,5.73,15.753.833.000,0,15.753.833.000,ABEV3
3,30/06/2023,12120.00,133290.00,1320.00,2670.0,3990.0,-8120.0,85070.00,5.40,15.753.833.000,0,15.753.833.000,ABEV3
4,31/03/2023,12210.00,135470.00,1140.00,2650.0,3790.0,-8420.0,85000.00,5.40,15.753.833.000,0,15.753.833.000,ABEV3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,31/12/2000,440.78,7540.00,451.05,3830.0,4290.0,3850.0,1060.00,0.00,554.722.817.000,535.892.517.000,1.090.615.334.000,BRAP4
92,30/09/2000,NaN,897.19,NaN,NaN,NaN,NaN,888.40,0.00,554.722.817.000,535.892.517.000,1.090.615.334.000,BRAP4
93,30/06/2000,NaN,947.47,NaN,NaN,NaN,NaN,942.91,0.00,554.722.817.000,535.892.517.000,1.090.615.334.000,BRAP4
94,31/03/2000,NaN,997.13,NaN,NaN,NaN,NaN,997.13,0.00,554.722.817.000,535.892.517.000,1.090.615.334.000,BRAP4


In [5]:
from selenium.webdriver.chrome.service import Service
refazer_comeco = True
chrome_driver_path = "/usr/bin/chromedriver"
service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    #'VVBR3', 'BTOW3','ENBR3',GNDI3,IGTA3
    'BRFS3', 'BRKM5',  'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
    'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
     'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4',  'GOAU4', 'GOLL4', 'HAPV3',
     'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
    'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3','SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
    'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'WIZC3','YDUQ3','SOMA3','BPAC11','MDIA3'
]